In [4]:
import pandas as pd
import numpy as np
import json

In [5]:
pd.set_option('display.max_columns', 5000)

In [1]:
def comparison(playerid, matchId, file, cols):
    total=pd.read_csv(file)
    m=total[cols].mean()
    print('The mean:')
    print(m)
    total=total[total['playerId']==playerid]
    total=total[total['matchId']==matchId]
    path='data/events/'+str(matchId)+'.json'
    with open(path, 'r', encoding='utf-8') as f:
        rd = f.read()
    rd=rd[28:-2]
    obj = json.loads(rd)
    dfw = pd.json_normalize(obj)
    dfw=dfw[dfw['player.id']==playerid]
    print('player:', dfw['player.name'].iloc[0])
    print(total[cols])
    

In [2]:
def get_player(playerid, matchId):
    path='data/events/'+str(matchId)+'.json'
    with open(path, 'r', encoding='utf-8') as f:
        rd = f.read()
    rd=rd[28:-2]
    obj = json.loads(rd)
    dfw = pd.json_normalize(obj)
    dfw=dfw[dfw['player.id']==playerid]
    return dfw['player.name'].iloc[0]

In [33]:
def bestplayers(df):
    for i in range (55):
        id=df['playerId'].iloc[i]
        matchid=df['matchId'].iloc[i]
        player=get_player(id,matchid)
        print('matchid:',matchid)
        print('playerid:', id)
        print(player)

In [7]:
loss_list= [
    'total.losses',
    'total.ownHalfLosses',
    'total.dangerousOwnHalfLosses'
]

In [8]:
def preprocessdf(mean, std, position):
    meandf=pd.read_csv(mean)
    stddf=pd.read_csv(std)
    pdf=pd.read_csv(position)
    meandf=meandf[meandf.columns[1:]]
    stddf=stddf[stddf.columns[1:]]
    '''
    pdf=pdf[pdf.columns[1:]]
    '''
    keep_columns=meandf.columns
    calc_col = [col for col in pdf.columns if col.startswith('total.') and col != 'total.minutesOnField']
    calc_filtered = [col for col in calc_col if col not in keep_columns]
    cpdf=pdf.drop(columns=calc_filtered)
    for w in loss_list:
        if(w in keep_columns):
            cpdf[w]*=-1
    return cpdf, meandf, stddf

In [9]:
def resultcalculation(calc,mean,std, weight):
    t0=calc.copy()
    t1=calc.copy()
    t2=calc.copy()
    t3=calc.copy()
    
    t0[mean.columns]-=mean.loc[0]
    t1[mean.columns]-=mean.loc[1]
    
    t0[mean.columns]/=std.loc[0]
    t1[mean.columns]/=std.loc[1]
    '''
    t2[mean.columns]-=mean.loc[2]
    t2[mean.columns]/=std.loc[2]
    t3[mean.columns]-=mean.loc[3]
    t3[mean.columns]/=std.loc[3]
    '''
    vec=np.array(weight)
    calc_col = [col for col in calc.columns if col.startswith('total.') and col != 'total.minutesOnField']
    statsarray0=t0[calc_col].to_numpy()
    statsarray1=t1[calc_col].to_numpy()
    statsarray2=t2[calc_col].to_numpy()
    statsarray3=t3[calc_col].to_numpy()
    res0=np.dot(statsarray0, vec)
    res1=np.dot(statsarray1, vec)
    res2=np.dot(statsarray2, vec)
    res3=np.dot(statsarray3, vec)
    return res0, res1, res2

In [10]:
meanpath='mean_std/mean_wi2v2clustering.csv'
stdpath='mean_std/std_wi2v2clustering.csv'
playerpath='gamesfromplayers/vargas_games.csv'

In [ ]:
pd.read_csv(meanpath)

In [12]:
calcdf, meandf, stddf=preprocessdf(meanpath, stdpath, playerpath)

In [ ]:
calcdf

In [14]:
#weight vector for each player profile
iv_def_weights=[1,2,1,4,1,4,1,4,2,1,4,1,2,4,1,2,4,1,0,1,1]
iv_hyb_weights=[1,2,1,4,1,4,1,4,2,1,4,1,4,1,1,2,4,1,0,1,2]
iv_off_weights=[1,4,1,4,1,1,1,4,4,1,4,1,4,1,2,2,4,2,2,1,2]
wi_pla_weights=[1,1,4,1,1,4,2,1,4,2,4,4,2,4,4]
wi_dri_weights=[1,1,4,1,4,2,4,1,4,4,4,2,4,4,2]
wi_run_weights=[1,1,4,1,2,2,4,1,4,4,4,1,4,4,4]
st_run_weights=[1,0,2,2,4,2,1,4,4,4,4,1,4]
st_goa_weights=[1,0,2,2,4,2,4,4,4,2,2,1,4]
st_box_weights=[2,0,2,2,2,1,1,4,2,2,4,1,4]
st_fa9_weights=[2,0,2,2,4,2,4,4,4,2,2,2,4]
mi_dpl_weights=[1,4,0,1,4,4,1,4,1,2,1,1,2,4,4,1,1,2,4,0,2,4,0,1]
mi_opl_weights=[0,0,4,0,0,4,1,4,0,0,4,0,0,0,2,1,1,0,0,4,4,2,2,0]
mi_def_weights=[1,4,0,1,2,2,1,4,1,2,1,1,1,4,4,2,4,2,4,0,2,4,0,2]
mi_off_weights=[0,0,4,1,4,1,1,4,0,0,1,0,0,2,2,2,4,0,0,2,4,2,2,4]
lb_def_weights=[1,2,1,4,1,4,1,4,1,1,1,2,4,1,4,1,2,4,1,1,4,1,0,0,1,1]
lb_off_weights=[1,4,1,2,1,2,2,4,2,4,1,4,2,1,2,1,2,2,4,2,2,1,4,4,1,1]
lb_run_weights=[1,4,1,4,1,4,1,2,1,4,1,4,2,1,4,1,2,2,4,1,2,1,4,4,1,2]
unitlist=[1]*len(mi_def_weights)

In [45]:
featuredf=pd.DataFrame(columns=meandf.columns)
'''
featuredf.loc[0]=mi_opl_weights
featuredf.loc[1]=iv_hyb_weights
featuredf.loc[2]=iv_off_weights
'''
featuredf

,total.duels,total.duelsWon,total.defensiveDuels,total.defensiveDuelsWon,total.aerialDuels,total.aerialDuelsWon,total.passes,total.successfulPasses,total.crosses,total.successfulCrosses,total.forwardPasses,total.successfulForwardPasses,total.interceptions,total.defensiveActions,total.successfulDefensiveAction,total.looseBallDuels,total.looseBallDuelsWon,total.recoveries,total.opponentHalfRecoveries,total.losses,total.ownHalfLosses,total.dangerousOwnHalfLosses,total.xgAssist,total.progressiveRun,total.slidingTackles,total.successfulSlidingTackles


In [17]:
r0,r1,r2=resultcalculation(calcdf,meandf,stddf, wi_pla_weights)

In [18]:
#calcdf['dribbler']=r2

calcdf['playmaker']=r0
#calcdf['Res2def']=r2

In [93]:
calcdf.to_csv('rating_amdouni.csv' , index=False)

In [257]:
ranking=calcdf.sort_values(by='Res1def', ascending=False)

In [57]:
r=pd.read_csv('ranking/final/ranking_st_fa9.csv')

In [58]:
ranking=r.sort_values(by='fa9', ascending=False)

In [59]:
ranking=r.sort_values(by='fa9', ascending=False)
ranking=ranking.reset_index()

In [ ]:
ranking[ranking['playerId']==614755]

In [61]:
bestplayers(ranking)

matchid: 2800011
playerid: 101669
A. Dzyuba
matchid: 5414266
playerid: 614755
Z. Amdouni
matchid: 2799552
playerid: 8717
H. Kane
matchid: 2799965
playerid: 97592
M. Dabbur
matchid: 2799578
playerid: 8717
H. Kane
matchid: 2799923
playerid: 3291
Paco Alcácer
matchid: 5414162
playerid: 273206
G. Rodrigues
matchid: 2799664
playerid: 118
M. Depay
matchid: 2800067
playerid: 14879
T. Pukki
matchid: 2799654
playerid: 3322
Cristiano Ronaldo
matchid: 2800021
playerid: 454072
A. Georgiou
matchid: 2800028
playerid: 101669
A. Dzyuba
matchid: 2799965
playerid: 20824
E. Zahavi
matchid: 2799941
playerid: 14703
M. Arnautović
matchid: 2799660
playerid: 135923
Gonçalo Paciência
matchid: 2799813
playerid: 15199
M. Gavranović
matchid: 2800039
playerid: 20479
F. Quagliarella
matchid: 2799667
playerid: 234710
C. Washington
matchid: 2800049
playerid: 8327
E. Džeko
matchid: 2799838
playerid: 282768
C. Itten
matchid: 5414229
playerid: 3327
Joselu
matchid: 2799587
playerid: 8717
H. Kane
matchid: 2799685
playerid

In [ ]:
path="norm_stats/adjustedstrikerstats.csv"
comparison(101669,2800011,path, meandf.columns)

In [259]:
ranking.to_csv('rank_fb_runweights.csv',index=False)